In [46]:
!conda info


     active environment : run_models
    active env location : C:\Users\mm507t\AppData\Local\anaconda3\envs\run_models
            shell level : 1
       user config file : C:\Users\mm507t\.condarc
 populated config files : C:\Users\mm507t\.condarc
          conda version : 24.1.2
    conda-build version : 24.1.2
         python version : 3.11.7.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=x86_64
                          __conda=24.1.2=0
                          __cuda=12.4=0
                          __win=0=0
       base environment : C:\Users\mm507t\AppData\Local\anaconda3  (writable)
      conda av data dir : C:\Users\mm507t\AppData\Local\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaco

In [2]:
import os
from typing import Dict, List
import chromadb
import yaml
import chainlit as cl

from llama_index.core.callbacks.base import CallbackManager
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Settings, VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.llms import ChatMessage

C:\Users\mm507t\AppData\Local\anaconda3\envs\run_models\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
TV_PKL_PATH = None
CHUNK_SIZE = None
CHUNK_OL = None
context_window = None
embed_model_name = None
nodes_path = None
bm25_pkl = None
persist_dir = None
data_read_path = None
CHUNK_SIZE = 1024
CHUNK_OL = 50
context_window = 5000
# embed_model_name : "BAAI/bge-small-en-v1.5"
persist_dir = "vector_store"
file_paths= 'WCFData'

In [5]:
llm = Ollama(model="phi3:latest",
              request_timeout=500.0 )
              # context_window=context_window)
Settings.llm = llm

In [6]:
embed_model_name = "sartifyllc/MultiLinguSwahili-serengeti-E250-nli-matryoshka"
truncate_dim = 64
Settings.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

2024-08-01 15:23:00 - Load pretrained SentenceTransformer: sartifyllc/MultiLinguSwahili-serengeti-E250-nli-matryoshka
2024-08-01 15:23:05 - 2 prompts are loaded, with the keys: ['query', 'text']


In [7]:
Settings.context_window = context_window
Settings.chunk_overlap = 50
Settings.chunk_size = 1024
Settings.node_parser = TokenTextSplitter(chunk_size=Settings.chunk_size, chunk_overlap=Settings.chunk_overlap, separator=" ")

In [8]:
def ingest_docs(input_files, storage_path=persist_dir, callback = CallbackManager()):
    # Settings.callback_manager = callback
    docs = SimpleDirectoryReader(input_dir=input_files).load_data()
    if not os.path.exists(storage_path):
        os.makedirs(storage_path)
    db = chromadb.PersistentClient(path=storage_path)
    chroma_collection = db.get_or_create_collection('quickstart')
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    vector_index = VectorStoreIndex.from_documents(documents=docs, storage_context=storage_context, embed_model= Settings.embed_model)
    print('docs_ingested')
    return vector_index

In [9]:
def load_components(storage_path=persist_dir):
    # service_context = ServiceContext.from_defaults(llm=Settings.llm, embed_model=Settings.embed_model)
    db = chromadb.PersistentClient(path=storage_path)
    chroma_collection = db.get_collection('quickstart')
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    # query_engine = index.as_query_engine(streaming=True, similarity_top_k=3)

    return index

In [10]:
index = ingest_docs(file_paths, storage_path=persist_dir)

2024-08-01 15:23:07 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Batches: 100%|██████████| 1/1 [00:04<00:00,  4.41s/it]


docs_ingested


In [11]:
# index = load_components(persist_dir)
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3)

In [17]:
print(query_engine.query("Is the Medical Services Portal available in multiple languages?").print_response_stream())

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.95it/s]


2024-08-01 15:28:18 - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 Yes, there are services available in both English and Kiswahili as per the provided details.None


In [19]:
print(query_engine.query("Wat is WCF?").print_response_stream())

Batches: 100%|██████████| 1/1 [00:00<00:00, 25.49it/s]


2024-08-01 15:29:52 - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 WCF (Workers Compensation Fund) is an organization established by law to provide insurance coverage for workers in Tanzania against work-related injuries or diseases as per Surat al Baiti [Chapter] Al Fadilah, verses 263. It was created on July 1st, 2015 following the issuance of Law No. 10/2011 by Parliament to ensure that every employee who suffers from a work-related injury or disease is compensated for their loss and medical expenses without having to prove fault against employers (Claim Review Number). WCF's aim aligns with Islamic principles, emphasizing fairness towards workers in need. The Fund operates as part of the Union Government under Section 18A(2) & B(3) of Law No.5/2016 and is an autonomous body that works independently from employers or any political groups to manage compensation for work-related injuries without bias towards different sectors, genders, religions, ethnicities, 

In [45]:
import gradio as gr

def query(text):

    return query_engine.query(text).print_response_stream()

def interface(text):
    # z = query(text)


    return query(text)

with gr.Blocks(theme=gr.themes.Glass().set(block_title_text_color= "black", body_background_fill="black", input_background_fill= "black", body_text_color="white")) as demo:
    
    gr.Markdown("<style>h1 {text-align: center;display: block;}</style><h1>Worker Compensation Fund</h1>")
        
    with gr.Row():
        input_text = gr.Textbox(label='Enter your query here')
        
    with gr.Row():
        output_text = gr.Textbox(label='Response')
        
    input_text.submit(fn=interface, inputs=input_text, outputs=output_text)
                      
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7871
2024-08-01 16:09:39 - HTTP Request: GET http://127.0.0.1:7871/startup-events "HTTP/1.1 200 OK"
2024-08-01 16:09:39 - HTTP Request: HEAD http://127.0.0.1:7871/ "HTTP/1.1 200 OK"
2024-08-01 16:09:39 - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-08-01 16:09:40 - HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"
2024-08-01 16:09:40 - HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_windows_amd64.exe "HTTP/1.1 200 OK"

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.40it/s]


2024-08-01 16:10:06 - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
 WCF refers to an entity established in accordance with a specific section of legislation aimed at ensuring fair compensation for employees who suffer occupational injuries or contract diseases resulting from their work duties within Tanzania's public and private sectors. Its main objective is the provision of adequate benefits, such as medical aid services and payment options based on permanent or temporary disability occurrences due to employment-related incidents.